## Spooky Author Identification exercise

Spooky authors dataset. Mоже да се изтегли от тук:
https://www.kaggle.com/c/spooky-author-identification

#### Изводи

След направените анализи в/у feature-те по време на лекцията на курса можем да стигнем до следните наблюдения и изводи:
- ...
- ...

Също така можем да опитаме някакъв feature engineering, "почиставне" и обработка на данните:
- ...
- ...

### Стратегия
- ...
- ...

#### Започваме

...

In [7]:
%config IPCompleter.greedy=True

In [8]:
!pip install numpy scipy matplotlib ipython scikit-learn pandas pillow mglearn

    100% |████████████████████████████████| 542kB 944kB/s ta 0:00:01
  Running setup.py bdist_wheel for mglearn ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/79/8b/2b/17dcfb9c9b044b216a58daea9787a0637cb1ffc5b4c2a78e50
Successfully built mglearn


...

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import mglearn

from IPython.display import display

%matplotlib inline

...

In [10]:
import pandas as pd
train = pd.read_csv("data/spooky-authors/train.zip", index_col=['id'])
test = pd.read_csv("data/spooky-authors/test.zip", index_col=['id'])
sample_submission = pd.read_csv("data/spooky-authors/sample_submission.zip", index_col=['id'])

print(train.shape, test.shape, sample_submission.shape)
print(set(train.columns) - set(test.columns))

(19579, 2) (8392, 1) (8392, 3)
{'author'}


In [11]:
train.head(5)

,text,author
id,,
id26305,"This process, however, afforded me no means of...",EAP
id17569,It never once occurred to me that the fumbling...,HPL
id11008,"In his left hand was a gold snuff box, from wh...",EAP
id27763,How lovely is spring As we looked from Windsor...,MWS
id12958,"Finding nothing else, not even gold, the Super...",HPL


...

In [16]:
!pip install nltk

    100% |████████████████████████████████| 1.2MB 558kB/s ta 0:00:01
  Running setup.py bdist_wheel for nltk ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/18/9c/1f/276bc3f421614062468cb1c9d695e6086d0c73d67ea363c501
Successfully built nltk


In [23]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [67]:
params = {"features__ngram_range": [(1,1), (1,2), (1,3)],
                      "features__analyzer": ['word'],
                      "features__max_df": [1.0, 0.9, 0.8, 0.7, 0.6, 0.5],
                      "features__min_df": [2, 3, 5, 10],
                      "features__lowercase": [False, True],
                      "features__stop_words": [None, stopwords],
                      "features__token_pattern": [r'\w+|\,', None],
                      "clf__alpha": [0.01, 0.1, 0.5, 1, 2]
                    }

In [64]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import log_loss

In [65]:
def report(results, n_top=5):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [68]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

# from nltk.corpus import stopwords
# stopset = set(stopwords.words('english'))

# from sklearn.ensemble import VotingClassifier

pipeline = Pipeline([
    ('features', TfidfVectorizer(
#         ngram_range=(1,2), min_df=2, 
#                                 max_df=0.8, lowercase=False, 
#                                 token_pattern=r'\w+|\,', stop_words=stopset
                                )),
    ('clf', MultinomialNB(
#         alpha=0.01
    )),
])

# print(pipeline.named_steps['features'])

# print(cross_val_score(pipeline, train.text, train.author, cv=3, n_jobs=3))
# print(cross_val_score(pipeline, train.text, train.author, cv=3, n_jobs=3, 
#                       scoring='neg_log_loss'))

random_search = RandomizedSearchCV(pipeline, param_distributions=params, 
                                       scoring='neg_log_loss',
                                       n_iter=20, cv=3, n_jobs=4)

random_search.fit(train.text, train.author)
report(random_search.cv_results_)

TypeError: catching classes that do not inherit from BaseException is not allowed

In [38]:
pipeline = pipeline.fit(train.text, train.author)

In [40]:
print(pipeline.predict_proba(test[:10].text))

[[  1.98283890e-02   3.81339586e-03   9.76358215e-01]
 [  9.96057571e-01   3.11816161e-03   8.24267151e-04]
 [  7.44699788e-03   9.90084289e-01   2.46871320e-03]
 [  8.24766674e-01   1.73950931e-01   1.28239521e-03]
 [  5.33013756e-01   3.65558628e-01   1.01427616e-01]
 [  9.14797902e-01   8.29368343e-02   2.26526379e-03]
 [  8.60877068e-01   1.35091168e-01   4.03176395e-03]
 [  1.20976104e-02   1.99588590e-01   7.88313799e-01]
 [  9.92430401e-01   7.55044856e-03   1.91502121e-05]
 [  7.66137344e-01   9.48152515e-02   1.39047405e-01]]


In [39]:
test_predictions = pipeline.predict_proba(test.text)

In [41]:
print(pipeline.classes_)

['EAP' 'HPL' 'MWS']


In [42]:
submit_file = pd.DataFrame(test_predictions, columns=['EAP', 'MWS', 'HPL'], index=test.index)
submit_file.head(10)

,EAP,MWS,HPL
id,,,
id02310,0.019828,0.003813,0.976358
id24541,0.996058,0.003118,0.000824
id00134,0.007447,0.990084,0.002469
id27757,0.824767,0.173951,0.001282
id04081,0.533014,0.365559,0.101428
id27337,0.914798,0.082937,0.002265
id24265,0.860877,0.135091,0.004032
id25917,0.012098,0.199589,0.788314
id04951,0.992430,0.007550,0.000019


In [43]:
submit_file.to_csv("data/spooky-authors/predictions.csv")